In [39]:
import os
archivos_entrenamiento = os.listdir("dataset-nlp-plagio-utn/")
set([os.path.splitext(archivo)[1] for archivo in archivos_entrenamiento]) # obtengo las extensiones de los archivos

{'.doc', '.docx', '.pdf', '.pptx'}

In [51]:
class ArchivoEntrenamiento:
    def __init__(self, nombre, extension, texto):
        self.nombre = nombre
        self.extension = extension
        self.texto = texto

In [52]:
from tika import parser
from nltk import word_tokenize

def convertir_documento_a_txt(archivo):
    raw = parser.from_file(archivo)
    return raw['content']

#word_tokenize(convertir_pdf_txt("dataset-nlp-plagio-utn/Domótica_Final.pptx"))

In [53]:
def convertir_archivo_a_txt(archivo):
    archivo_nombre, archivo_extension = os.path.splitext(archivo)
    if(archivo_extension != ".pptx"):
        archivo_txt = convertir_documento_a_txt(archivo)
        return ArchivoEntrenamiento(archivo_nombre, archivo_extension, archivo_txt)
    
#convertir_archivo_a_txt("dataset-nlp-plagio-utn/MKT 2016 - Alan Szpigiel - TP4.pdf").nombre

In [55]:
archivos_entrenamiento_txt = [convertir_archivo_a_txt("dataset-nlp-plagio-utn/" + archivo) for archivo in archivos_entrenamiento]

In [79]:
from nltk import sent_tokenize # divide en oraciones
from nltk import re
sent_tokenize(archivos_entrenamiento_txt[1].texto.strip())[10]

'7.- Cuáles son las ventajas para la empresa de la personalización masiva?'